In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch.nn as nn
import torch 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from training_functions import train_model, test_model
from sklearn.metrics import mean_squared_error

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from food_price import preprocess, lstm

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


Data Processing

In [6]:
df = pd.read_csv('data/food_price_inflation.csv')
print(df.shape)

(59839, 4)


In [7]:
# we keep only data in France 


print(df.shape)

(59839, 4)


Training

In [17]:
# preprocessing 
train_set, val_set, test_set, scaler_X, scaler_y = preprocess(
    df, 
    split_ratio_train_test=0.8, 
    split_ratio_train_valid=0.8,
    device=device
)
print(train_set.tensors[0].shape, val_set.tensors[0].shape, test_set.tensors[0].shape)
input_size = train_set.tensors[0].shape[2]

torch.Size([168, 20, 26]) torch.Size([27, 20, 26]) torch.Size([39, 20, 26])


In [18]:
# training's hyperparameters
model = lstm(input_size=input_size, mode = 'relu').to(device)
criterion = nn.MSELoss()
learning_rate = 0.001
num_epochs = 200
batch_size = 32
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loader = DataLoader(train_set, batch_size=batch_size)
val_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)


In [19]:
history = train_model(model, criterion, optimizer, num_epochs,train_loader, val_loader,device)

Epoch 1/200, Training Loss: 5.4951, Validation Loss: 4.1742
Epoch 2/200, Training Loss: 3.0162, Validation Loss: 2.0145
Epoch 3/200, Training Loss: 2.2811, Validation Loss: 1.5460
Epoch 4/200, Training Loss: 1.7688, Validation Loss: 2.3842
Epoch 5/200, Training Loss: 1.6427, Validation Loss: 3.6133
Epoch 6/200, Training Loss: 1.2484, Validation Loss: 3.2437
Epoch 7/200, Training Loss: 1.2730, Validation Loss: 2.3850
Epoch 8/200, Training Loss: 0.8901, Validation Loss: 2.9587
Epoch 9/200, Training Loss: 0.8451, Validation Loss: 3.2010
Epoch 10/200, Training Loss: 0.9686, Validation Loss: 2.9207
Epoch 11/200, Training Loss: 0.7120, Validation Loss: 3.6026
Epoch 12/200, Training Loss: 0.6386, Validation Loss: 4.0363
Epoch 13/200, Training Loss: 0.9114, Validation Loss: 3.3846
Epoch 14/200, Training Loss: 0.8526, Validation Loss: 3.5438
Epoch 15/200, Training Loss: 0.6928, Validation Loss: 3.8060
Epoch 16/200, Training Loss: 0.6012, Validation Loss: 3.8349
Epoch 17/200, Training Loss: 0.65

In [20]:
print(test_model(model, test_loader, criterion, device))

75.15329090257485
